In [3]:
# 10294.532652920545, 21.991946458165227  (trained on normal env)
# 10291.333417544305, 21.18661703314784 
# 5866.360775249422, 56.68289779522459 - base policy ( ithink)

# 10884.874199650516, 29.484380976179228 - every step

In [4]:
# 25 steps
# with degradation
# with failure
# no 2 in action
# changed order
# random start withdegradation (85 to 100)
# export cap, decreasing fit
# fixed IRR calculation
# energy prices and FIT in the obs space (normaliosed)
# if replaced the bos is not calculated in 
# efficency impr incl
# CAPEX incl, and in the obs space
# TOU included
# , simplified, optimised
# no npv and irr in the train env
# pv costs and accual intwerest sadded to the mix
#  not loading data as pd dataframes
# changed action space

In [1]:
# imports
import pandas as pd
import numpy as np
import numpy_financial as npf
import random  
import matplotlib.pyplot as plt
import time

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO, DQN, A2C
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib import RecurrentPPO

from environment_fx import calculate_import_export, test1, test2, test3, evaluate1, evaluate2, basepolicy

import gymnasium as gym
from gymnasium import spaces

import torch
import torch as th
from torch import nn
import torch.nn as nn

In [2]:
# import and modify data

# Assuming the file is a CSV and specifying the correct path and filename
file_path = r"C:\Users\kubaw\Desktop\DELFT\THESIS\CH5"

# Use pandas to read the CSV file
AC_OUTPUT = pd.read_csv(file_path + "/AC_OUTPUT_JA")
elec_df = pd.read_csv(file_path + "/hourly_consumption_gemany.csv")
import_price = pd.read_csv(file_path + "/electricity_tariff.csv")

#elec_df = elec_df * 1000
elec_df = elec_df.drop('HourOfYear', axis=1)

elec_df['hour_of_day'] = np.arange(8760) % 24
elec_df['day_of_week'] = np.arange(8760) // 24 % 7  # 0 is Monday, 6 is Sunday

# Define rates
peak_rate = 1.45
normal_rate = 1
off_peak_rate = 0.85

# Function to determine rate based on hour and day
def determine_rate(hour, day):
    if day < 5:  # Monday to Friday
        if 16 <= hour < 21:  # 4pm to 9pm
            return peak_rate
        elif 6 <= hour < 10:  # 7am to 9am and 10am to 3pm
            return normal_rate
        else:  # Off-peak times
            return off_peak_rate
    else:  # Weekend
        if 16 <= hour < 21:  # 4pm to 9pm
            return normal_rate
        else:  # Off-peak times
            return off_peak_rate
    
# Apply the function to each row to determine the rate
elec_df['rate'] = elec_df.apply(lambda row: determine_rate(row['hour_of_day'], row['day_of_week']), axis=1)

import_price_df = import_price.drop(columns=['x'])
import_price_df = import_price_df[:-26]

train_cols = random.sample(list(import_price_df.columns), 7000)
import_price_train = import_price_df[train_cols]
test_cols = [col for col in import_price_df.columns if col not in train_cols]
import_price_test = import_price_df[test_cols]

Eff = pd.read_csv(file_path + "/Efficency_impr")
Eff = (Eff)/100 + 1

CAPEX = pd.read_csv(file_path + "/CAPEX_JA.csv")
CAPEX_JA = (CAPEX[:26]) * 1.3


train_cols_CAPEX = random.sample(list(CAPEX_JA.columns), 7000)
test_cols_CAPEX = [col for col in CAPEX_JA.columns if col not in train_cols_CAPEX]

CAPEX_JA_train = CAPEX_JA[train_cols_CAPEX]
CAPEX_JA_test = CAPEX_JA[test_cols_CAPEX]

train_cols_Eff = random.sample(list(Eff.columns), 7000)
test_cols_Eff = [col for col in Eff.columns if col not in train_cols_Eff]

Eff_train = Eff[train_cols_Eff]
Eff_test = Eff[test_cols_Eff]

AC_OUTPUT_arr = (np.array(AC_OUTPUT.T)).flatten()

Eff_train_arr = np.array(Eff_train.T)
Eff_test_arr = np.array(Eff_test.T)

CAPEX_JA_train_arr = np.array(CAPEX_JA_train.T)
CAPEX_JA_test_arr = np.array(CAPEX_JA_test.T)

elec_consum_arr = np.array(elec_df["Consumption"])
import_price_rate = np.array(elec_df["rate"])

import_price_train_arr = np.array(import_price_train.T)
import_price_test_arr = np.array(import_price_train.T)

In [3]:
class TrainEnvironment(gym.Env):
    def __init__(self, AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_tariff, efficency, CAPEX):
        
        # Price per watthour
        self.import_price_df = import_tariff
        self.import_price_at_zero = np.float32(0.00035)
        self.import_price_rate = import_price_rate
        
        # Energy Balance
        self.AC_OUTPUT = AC_OUTPUT_arr
        self.elec_df = elec_consum_arr
        self.max_export = 4000
        self.number_of_panels = 24
        
        # Degradation
        self.deg_mu = 1.055
        self.deg_std = 0.555
        
        # Efficency Development
        self.efficency_develop_df = efficency
        self.efficency_at_zero = 1.0
        
        # Costs
        self.power_at_zero = 415
        self.cost_per_Wp_df_at_zero = 0.69
        self.cost_per_Wp_df = CAPEX
        self.initial_other_costs = 150
        
        self.operational_cost = 16.8
        #self.budget_constraint = 1500
        
        self.loan_interest_rate = 1.06
        self.normal_interest_rate = 1.02
                        
        # Spaces and length
        self.action_space = spaces.Discrete(self.number_of_panels + 1)
        self.observation_space = spaces.Box(0, 1.25, shape=(self.number_of_panels + 7,))
        self.episode_len = 25
        self.months_per_timestep = 12
        
    def _get_obs(self):
        
        return self.observation
    
    def calculate_import_export(self, AC_OUTPUT, elec_df, export_price, import_price):
        
        """
        Calculate the annual Wh of energy exported to the grid (exported) and saved (minimised)
        """
        
        AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT 

        exported = (AC_OUTPUT_tot - self.elec_df).clip(min=0, max = self.max_export)        
        export_revenue = (export_price * exported).sum()

        
        minimised = AC_OUTPUT_tot - exported 
        minimised_revenue = (minimised * (self.import_price_rate * import_price)).sum()
        

        return export_revenue, AC_OUTPUT_tot, minimised_revenue
    
    def reset(self, seed=None):
        
        """
        Reset the environment to the original state at t=1
        """
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        # Panels
        self.init_obs = np.random.uniform(0, 1, size=self.number_of_panels).astype(np.float32)
        self.init_obs = np.where(self.init_obs < 0.5, 0.0, np.random.uniform(0.85, 1.0, size=self.number_of_panels))

        # Combine all initialization into a single step for efficiency
        self.import_price_at_zero_norm = (self.import_price_at_zero - 0.00022499) / (0.0020798 - 0.00022499)
        self.FiT_at_zero_norm = (self.import_price_at_zero - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
        self.efficency_at_zero_norm = (self.efficency_at_zero - 0.999) / (1.156 - 0.999)
        self.panel_cost_and_inverter_at_zero_norm = (self.cost_per_Wp_df_at_zero - 0.254) / (1.12 - 0.254)
        
        self.current_budget_constraint = np.random.randint(0, 750)
        self.next_step_budget_constraint = 0
        
        
        # Complete observation initialization in one go
        self.observation = np.concatenate([
            self.init_obs,
            [self.import_price_at_zero_norm, self.FiT_at_zero_norm, self.efficency_at_zero_norm, 
             self.panel_cost_and_inverter_at_zero_norm, 0., 0., 0.]
        ]).astype(np.float32)

        self.previous_observation = self.observation.copy()

        # RANDOM IMPORT PRICE
        self.random_import_price = self.import_price_df[np.random.choice(self.import_price_df.shape[0])] 

        # RANDOM EFFICENCY
        self.random_efficency_develop = self.efficency_develop_df[np.random.choice(self.efficency_develop_df.shape[0])]   
        
        # RANDOM COST PER WP
        self.random_cost_per_Wp = self.cost_per_Wp_df[np.random.choice(self.cost_per_Wp_df.shape[0])]   
        
        
        self.episode_len = 25  
    
        info = {}
        
        # RESET BALANCES
        self.fin_balance_tot = 0
        self.reward_tot = 0
        self.env_balance_tot = 0
        self.produced = 0
        self.other_costs = 0
        self.FiT = 0.0004
        self.next_FiT = 0.0004

        self.total_cash_flow = []
        self.annual_cash_flow = 0
                
        self.due_loans = [0, 0, 0, 0] 
        self.current_interest = 0
        self.step_total_interest = 1
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
        self.resale_values = array_of_zeros = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.two_year_ago_interest = 0
        self.first_year_interest = []
        self.second_year_interest = [0]
        self.third_year_interest = [0, 0]
        self.fourth_year_interest = [0, 0, 0]
        self.next_year_total = 0
        
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
    
        return self.observation, info
    
    def calculate_resale(self, initial_panel_cost, indices):
        
        self.resale_values[indices] = initial_panel_cost
        
        self.resale_values = self.resale_values * 0.85
        
        for count, i in enumerate(self.broke):
            if i == 1:
                self.resale_values[count] = 0
        
        resale_step = self.resale_values[indices].sum()
        
        return resale_step
    
    def calculate_panel_inv_cost(self, cost_per_Wp):
        
        PW_ep = self.efficency_develop * self.power_at_zero
        
        panel_cost_and_inverter = PW_ep * cost_per_Wp
        
        return panel_cost_and_inverter
        
    def calculate_penalty(self, current_step, annual_expense):
              
        year = 25 - current_step
        
        if year > 0:
            self.current_budget_constraint = self.next_step_budget_constraint    
            
        
        self.current_interest = self.next_year_total
        annual_expense = (-annual_expense)
        value = 0 
        loan = 0
        annual_interest = 0

        if annual_expense > self.current_budget_constraint:
            loan = (self.current_budget_constraint - annual_expense)
            value = annual_expense / self.current_budget_constraint
            periods = 2 if value < 2 else 3 if value < 3 else 4

            annual_interest = loan / periods
            interest_multiplier = 1

            for i in range(4):
                if i < periods:
                    self.due_loans[i] = annual_interest * interest_multiplier
                    interest_multiplier *= self.loan_interest_rate
                else:
                    self.due_loans[i] = 0
        else:
             self.due_loans = [0, 0, 0, 0]
    
        self.first_year_interest.append(self.due_loans[0])
        self.second_year_interest.append(self.due_loans[1])
        self.third_year_interest.append(self.due_loans[2])
        self.fourth_year_interest.append(self.due_loans[3])
    
    
        self.next_year_total = self.first_year_interest[year] + self.second_year_interest[year] + self.third_year_interest[year] + self.fourth_year_interest[year]
        
        self.next_step_budget_constraint = np.random.randint(0, 750) * self.step_total_interest
        current_budget_observation = (self.next_step_budget_constraint - 0 * self.step_total_interest) / (750 * self.step_total_interest - 0 * self.step_total_interest) 
        self.observation[self.number_of_panels + 6] = current_budget_observation
                
        return self.current_interest, self.due_loans, self.next_year_total
        
    def calculate_total_CAPEX(self, action_step, panel_cost_and_inverter):
        """
        Calculate CAPEX each step in a vectorized manner.
        """
        BOS = panel_cost_and_inverter * 0.55
        number_installed = int(np.sum(action_step))

        # Calculate costs from module and inverter
        panel_cost_and_inverter_step = panel_cost_and_inverter * number_installed

        # Calculate other installation costs
        if number_installed == 0:
            other_costs = 0
        elif number_installed == 1:
            other_costs = self.initial_other_costs * self.step_total_interest
        else:
            discounts = 0.9 ** np.arange(number_installed)
            other_costs = (self.initial_other_costs * self.step_total_interest * discounts).sum()

        # Calculate BOS costs using vector operations
        is_new_installation = (self.previous_observation[:number_installed] == 0) & (action_step[:number_installed] == 1)
        is_replacement = (self.previous_observation[:number_installed] > 0) & (action_step[:number_installed] == 1)
        BOS_cost = np.sum(BOS * is_new_installation) + np.sum((BOS / 2) * is_replacement)

        # Sum total CAPEX
        total_CAPEX = panel_cost_and_inverter_step + BOS_cost + other_costs

        return total_CAPEX, panel_cost_and_inverter
        
    def failure(self, actions):
        
        beta = 3  # Shape parameter
        phi = 30  # Scale parameter  

        # Determine which panels are active based on the actions and previous observations.
        if self.episode_len == 24:
            active_panels = (self.observation[:self.number_of_panels] > 0.85)
        else:
            active_panels = (self.observation[:self.number_of_panels] == self.efficency_develop)

        # Calculate lifespan for all active panels at once
        lifespans = np.random.weibull(beta, self.number_of_panels) * phi
        lifespans = np.where(active_panels, lifespans, 0)  # Apply lifespan only to active panels

        # Adjust survival times based on episode length
        self.survival[:self.number_of_panels] = np.where(
            active_panels,
            np.abs(lifespans.astype(int)) + np.abs(self.episode_len - 25),
            self.survival[:self.number_of_panels]
        )

        return self.survival

    def calculate_FiT(self, episodes, import_price):
            
        self.FiT = import_price
            
        if episodes == 25:
            self.FiT = self.FiT
            
        elif episodes == 24 or episodes == 23:
            self.FiT = self.FiT * 0.64
            
        elif episodes == 22:
            self.FiT = self.FiT * 0.46
            
        elif episodes == 21:
            self.FiT = self.FiT * 0.55
            
        elif episodes < 20:
            self.FiT = self.FiT * 0.33
            
        elif episodes == 20:
            self.FiT = self.FiT * 0.37
            
        return self.FiT
                        
    def step(self, action):
        
        """
        defines actions, reward etc.
        """
        
        # RESET THE ANNUAL BALANCES
        self.total_CAPEX = 0
        self.pv_costs = 0
        self.fin_balance = 0
        self.number_installed = 0
        current_penalty = 0
        self.other_costs = 0
        next_step_penalty = 0
        self.step_total_interest = self.step_total_interest * self.normal_interest_rate
        current_operational_costs = self.operational_cost * self.step_total_interest
        
        
        self.cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
        self.import_price = self.random_import_price[abs(self.episode_len - 25)]
        self.efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
           
        self.panel_cost_and_inverter = self.calculate_panel_inv_cost(self.cost_per_Wp)
        FiT = self.calculate_FiT(self.episode_len, self.import_price)
        
        reward = 0   
        actions_step = np.random.rand(8)
        
        # Find indices of the lowest 'action' values in previous_observation
        indices = np.argsort(self.previous_observation[:self.number_of_panels])[:action]

        # Replace these indices in the observation with efficiency_develop
        self.observation[:self.number_of_panels][indices] = self.efficency_develop

        # Copy over the other values from previous_observation to observation
        mask = np.ones(len(self.previous_observation[:self.number_of_panels]), dtype=bool)
        mask[indices] = False
        self.observation[:self.number_of_panels][mask] = self.previous_observation[:self.number_of_panels][mask]

        replaced_panels = np.zeros(len(self.previous_observation[:self.number_of_panels]), dtype=int)
        replaced_panels[indices] = 1

        instaltion = (self.observation[:self.number_of_panels] > 0).astype(int)
        self.pv_costs -= instaltion.sum() * current_operational_costs

        actions_step = np.array(replaced_panels)

            
        if action > 0:
            step_CAPEX, panel_cost_and_inverter = self.calculate_total_CAPEX(actions_step, self.panel_cost_and_inverter)
            self.pv_costs -= step_CAPEX
            
        else:
            panel_cost_and_inverter = 0
                
        next_observation = self._get_obs()

        
        # Calculate the Reslae value
        resale = self.calculate_resale(panel_cost_and_inverter, indices) #  ***
        
        self.pv_costs += resale
 
        
        # CALCULATE THE BUDGET INTEREST
        current_penalty, due_loans, next_step_penalty = self.calculate_penalty(self.episode_len, self.pv_costs)

        
        # CALCULATE THE ENERGY YIELD
        exported_revenue, AC_OUTPUT_tot, minimised_revenue = self.calculate_import_export(self.AC_OUTPUT, 
                                                                          self.elec_df, FiT, self.import_price)        
        
        pv_costs_observation = - self.pv_costs / 10000
        self.observation[self.number_of_panels + 4] = pv_costs_observation
        
        next_step_penalty_observation = - next_step_penalty / 8000
        self.observation[self.number_of_panels + 5] = next_step_penalty_observation
        
        
        # CALCULATE STEP BALANCES
        self.fin_balance += self.pv_costs
        self.fin_balance += current_penalty
        self.fin_balance += float(exported_revenue + minimised_revenue)
        
        # CALCULATE TOTAL BALANCES
        self.fin_balance_tot += self.fin_balance                
        
        # SUBSTRACT 1 FOR TIMESTEP
        self.episode_len -= 1
        done = self.episode_len <= 0
        
        #reward = self.fin_balance_tot / 1000 if done else 0
        reward = self.fin_balance / 1000
        
        # FAILURE
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        survival = self.failure(actions_step)
        
        for c, p in enumerate(survival):
            
            if c < self.number_of_panels:

                if p - 1 <= abs(self.episode_len - 24):
                    self.broke[c] = 1
                    self.observation[c] = 0
        
        # DEGRADATION RATE
        # Applying degradation only to panels that are operational (above 0.1 efficiency)
        active_panels = self.observation[:self.number_of_panels] > 0.1
        degradations = np.random.normal(self.deg_mu, self.deg_std, size=self.number_of_panels) / 100
        self.observation[:self.number_of_panels][active_panels] -= degradations[active_panels]
        
        if not done: 
        
            self.next_cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
            self.next_import_price = self.random_import_price[abs(self.episode_len - 25)]
            self.next_efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
            next_FIT = self.calculate_FiT(self.episode_len, self.next_import_price)
        
            price_observation = (self.next_import_price - 0.00022499) / (0.0020798 - 0.00022499)
            self.observation[self.number_of_panels] = price_observation

            FiT_observation = (next_FIT - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
            self.observation[self.number_of_panels + 1] = FiT_observation

            eff_observation = (self.next_efficency_develop - 0.999) / (1.156 - 0.999)
            self.observation[self.number_of_panels + 2] = eff_observation

            cost_per_Wp_observation = (self.next_cost_per_Wp - 0.254) / (1.12 - 0.254) 
            self.observation[self.number_of_panels + 3] = cost_per_Wp_observation
        
        info = {"step financial balance (eur):": self.fin_balance,
               "total financial balance: (eur)": self.fin_balance_tot,
               "internal rate of return": 0,
               "current_interest": resale,
                "net present value": 0}
         
        
        self.previous_observation = self.observation.copy()
        
        return self.observation, reward, done, False, info

In [4]:
class TestEnvironment(gym.Env):
    def __init__(self, AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_tariff, efficency, CAPEX):
        
        # Price per watthour
        self.import_price_df = import_tariff
        self.import_price_at_zero = np.float32(0.00035)
        self.import_price_rate = import_price_rate
        
        # Energy Balance
        self.AC_OUTPUT = AC_OUTPUT_arr
        self.elec_df = elec_consum_arr
        self.max_export = 4000
        self.number_of_panels = 24
        
        # Degradation
        self.deg_mu = 1.055
        self.deg_std = 0.555
        
        # Efficency Development
        self.efficency_develop_df = efficency
        self.efficency_at_zero = 1.0
        
        # Costs
        self.power_at_zero = 415
        self.cost_per_Wp_df_at_zero = 0.69
        self.cost_per_Wp_df = CAPEX
        self.initial_other_costs = 150
        
        self.operational_cost = 16.8
        #self.budget_constraint = 1500
        
        self.loan_interest_rate = 1.06
        self.normal_interest_rate = 1.02
                        
        # Spaces and length
        self.action_space = spaces.Discrete(self.number_of_panels + 1)
        self.observation_space = spaces.Box(0, 1.25, shape=(self.number_of_panels + 7,))
        self.episode_len = 25
        self.months_per_timestep = 12
        
    def _get_obs(self):
        
        return self.observation
    
    def calculate_import_export(self, AC_OUTPUT, elec_df, export_price, import_price):
        
        """
        Calculate the annual Wh of energy exported to the grid (exported) and saved (minimised)
        """
        
        AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT 

        exported = (AC_OUTPUT_tot - self.elec_df).clip(min=0, max = self.max_export)        
        export_revenue = (export_price * exported).sum()

        
        minimised = AC_OUTPUT_tot - exported 
        minimised_revenue = (minimised * (self.import_price_rate * import_price)).sum()
        

        return export_revenue, AC_OUTPUT_tot, minimised_revenue
    
    def reset(self, seed=None):
        
        """
        Reset the environment to the original state at t=1
        """
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
            
        # Panels
        self.init_obs = np.random.uniform(0, 1, size=self.number_of_panels).astype(np.float32)
        self.init_obs = np.where(self.init_obs < 0.5, 0.0, np.random.uniform(0.85, 1.0, size=self.number_of_panels))

        # Combine all initialization into a single step for efficiency
        self.import_price_at_zero_norm = (self.import_price_at_zero - 0.00022499) / (0.0020798 - 0.00022499)
        self.FiT_at_zero_norm = (self.import_price_at_zero - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
        self.efficency_at_zero_norm = (self.efficency_at_zero - 0.999) / (1.156 - 0.999)
        self.panel_cost_and_inverter_at_zero_norm = (self.cost_per_Wp_df_at_zero - 0.254) / (1.12 - 0.254)
        
        self.current_budget_constraint = np.random.randint(0, 750)
        self.next_step_budget_constraint = 0
        
        
        # Complete observation initialization in one go
        self.observation = np.concatenate([
            self.init_obs,
            [self.import_price_at_zero_norm, self.FiT_at_zero_norm, self.efficency_at_zero_norm, 
             self.panel_cost_and_inverter_at_zero_norm, 0., 0., 0.]
        ]).astype(np.float32)

        self.previous_observation = self.observation.copy()

        # RANDOM IMPORT PRICE
        self.random_import_price = self.import_price_df[np.random.choice(self.import_price_df.shape[0])] 

        # RANDOM EFFICENCY
        self.random_efficency_develop = self.efficency_develop_df[np.random.choice(self.efficency_develop_df.shape[0])]   
        
        # RANDOM COST PER WP
        self.random_cost_per_Wp = self.cost_per_Wp_df[np.random.choice(self.cost_per_Wp_df.shape[0])]   
        
        
        self.episode_len = 25  
    
        info = {}
        
        # RESET BALANCES
        self.fin_balance_tot = 0
        self.reward_tot = 0
        self.env_balance_tot = 0
        self.produced = 0
        self.other_costs = 0
        self.FiT = 0.0004
        self.next_FiT = 0.0004
        self.resale_values = array_of_zeros = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        self.total_cash_flow = []
        self.annual_cash_flow = 0
                
        self.due_loans = [0, 0, 0, 0] 
        self.current_interest = 0
        self.step_total_interest = 1
        
        self.two_year_ago_interest = 0
        self.first_year_interest = []
        self.second_year_interest = [0]
        self.third_year_interest = [0, 0]
        self.fourth_year_interest = [0, 0, 0]
        self.next_year_total = 0
        
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
    
        return self.observation, info
    
    def calculate_resale(self, initial_panel_cost, indices):
        
        self.resale_values[indices] = initial_panel_cost
        
        self.resale_values = self.resale_values * 0.85
        
        for count, i in enumerate(self.broke):
            if i == 1:
                self.resale_values[count] = 0
        
        resale_step = self.resale_values[indices].sum()
        
        return resale_step
    
    def calculate_panel_inv_cost(self, cost_per_Wp):
        
        PW_ep = self.efficency_develop * self.power_at_zero
        
        panel_cost_and_inverter = PW_ep * cost_per_Wp
        
        return panel_cost_and_inverter
    
    def calculate_irr_and_npv(self, pv_cost, minimised_revenue, export_revenue, penalty):
                
        """
        Calculates total cash flow of the project needed for the internal rate of return
        """ 
        self.expences = 0
        self.annual_cash_flow = 0
        initial_cost = 0
        
        self.expences = pv_cost
        self.annual_cash_flow = self.expences + export_revenue + minimised_revenue + penalty
        initial_cost_q, x = self.calculate_total_CAPEX(self.init_obs, self.panel_cost_and_inverter)
        initial_cost = - initial_cost_q
        
        if self.episode_len == 24:
            self.total_cash_flow.append(initial_cost + self.annual_cash_flow) 
        else:
            self.total_cash_flow.append(self.annual_cash_flow) 
        
        return self.total_cash_flow
        
    def calculate_penalty(self, current_step, annual_expense):
              
        year = 25 - current_step
        
        if year > 0:
            self.current_budget_constraint = self.next_step_budget_constraint    
            
        
        self.current_interest = self.next_year_total
        annual_expense = (-annual_expense)
        value = 0 
        loan = 0
        annual_interest = 0

        if annual_expense > self.current_budget_constraint:
            loan = (self.current_budget_constraint - annual_expense)
            value = annual_expense / self.current_budget_constraint
            periods = 2 if value < 2 else 3 if value < 3 else 4

            annual_interest = loan / periods
            interest_multiplier = 1

            for i in range(4):
                if i < periods:
                    self.due_loans[i] = annual_interest * interest_multiplier
                    interest_multiplier *= self.loan_interest_rate
                else:
                    self.due_loans[i] = 0
        else:
             self.due_loans = [0, 0, 0, 0]
    
        self.first_year_interest.append(self.due_loans[0])
        self.second_year_interest.append(self.due_loans[1])
        self.third_year_interest.append(self.due_loans[2])
        self.fourth_year_interest.append(self.due_loans[3])
    
    
        self.next_year_total = self.first_year_interest[year] + self.second_year_interest[year] + self.third_year_interest[year] + self.fourth_year_interest[year]
        
        self.next_step_budget_constraint = np.random.randint(750, 2000) * self.step_total_interest
        current_budget_observation = (self.next_step_budget_constraint - 750 * self.step_total_interest) / (2000 * self.step_total_interest - 750 * self.step_total_interest) 
        self.observation[self.number_of_panels + 6] = current_budget_observation
                
        return self.current_interest, self.due_loans, self.next_year_total
        
    def calculate_total_CAPEX(self, action_step, panel_cost_and_inverter):
        """
        Calculate CAPEX each step in a vectorized manner.
        """
        BOS = panel_cost_and_inverter * 0.55
        number_installed = int(np.sum(action_step))

        # Calculate costs from module and inverter
        panel_cost_and_inverter_step = panel_cost_and_inverter * number_installed

        # Calculate other installation costs
        if number_installed == 0:
            other_costs = 0
        elif number_installed == 1:
            other_costs = self.initial_other_costs * self.step_total_interest
        else:
            discounts = 0.9 ** np.arange(number_installed)
            other_costs = (self.initial_other_costs * self.step_total_interest * discounts).sum()

        # Calculate BOS costs using vector operations
        is_new_installation = (self.previous_observation[:number_installed] == 0) & (action_step[:number_installed] == 1)
        is_replacement = (self.previous_observation[:number_installed] > 0) & (action_step[:number_installed] == 1)
        BOS_cost = np.sum(BOS * is_new_installation) + np.sum((BOS / 2) * is_replacement)

        # Sum total CAPEX
        total_CAPEX = panel_cost_and_inverter_step + BOS_cost + other_costs

        return total_CAPEX, panel_cost_and_inverter
        
    def failure(self, actions):
        
        beta = 3  # Shape parameter
        phi = 30  # Scale parameter  

        # Determine which panels are active based on the actions and previous observations.
        if self.episode_len == 24:
            active_panels = (self.observation[:self.number_of_panels] > 0.85)
        else:
            active_panels = (self.observation[:self.number_of_panels] == self.efficency_develop)

        # Calculate lifespan for all active panels at once
        lifespans = np.random.weibull(beta, self.number_of_panels) * phi
        lifespans = np.where(active_panels, lifespans, 0)  # Apply lifespan only to active panels

        # Adjust survival times based on episode length
        self.survival[:self.number_of_panels] = np.where(
            active_panels,
            np.abs(lifespans.astype(int)) + np.abs(self.episode_len - 25),
            self.survival[:self.number_of_panels]
        )

        return self.survival

    def calculate_FiT(self, episodes, import_price):
            
        self.FiT = import_price
            
        if episodes == 25:
            self.FiT = self.FiT
            
        elif episodes == 24 or episodes == 23:
            self.FiT = self.FiT * 0.64
            
        elif episodes == 22:
            self.FiT = self.FiT * 0.46
            
        elif episodes == 21:
            self.FiT = self.FiT * 0.55
            
        elif episodes < 20:
            self.FiT = self.FiT * 0.33
            
        elif episodes == 20:
            self.FiT = self.FiT * 0.37
            
        return self.FiT
                        
    def step(self, action):
        
        """
        defines actions, reward etc.
        """
        
        # RESET THE ANNUAL BALANCES
        self.total_CAPEX = 0
        self.pv_costs = 0
        self.fin_balance = 0
        self.number_installed = 0
        irr_fin = 0
        npv_fin = 0
        current_penalty = 0
        self.other_costs = 0
        next_step_penalty = 0
        self.step_total_interest = self.step_total_interest * self.normal_interest_rate
        current_operational_costs = self.operational_cost * self.step_total_interest
        
        
        self.cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
        self.import_price = self.random_import_price[abs(self.episode_len - 25)]
        self.efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
           
        self.panel_cost_and_inverter = self.calculate_panel_inv_cost(self.cost_per_Wp)
        FiT = self.calculate_FiT(self.episode_len, self.import_price)
        
        reward = 0   
        actions_step = np.random.rand(self.number_of_panels + 1)
        
        
        # Find indices of the lowest 'action' values in previous_observation
        indices = np.argsort(self.previous_observation[:self.number_of_panels])[:action]

        # Replace these indices in the observation with efficiency_develop
        self.observation[:self.number_of_panels][indices] = self.efficency_develop

        # Copy over the other values from previous_observation to observation
        mask = np.ones(len(self.previous_observation[:self.number_of_panels]), dtype=bool)
        mask[indices] = False
        self.observation[:self.number_of_panels][mask] = self.previous_observation[:self.number_of_panels][mask]

        replaced_panels = np.zeros(len(self.previous_observation[:self.number_of_panels]), dtype=int)
        replaced_panels[indices] = 1

        instaltion = (self.observation[:self.number_of_panels] > 0).astype(int)
        self.pv_costs -= instaltion.sum() * current_operational_costs

        actions_step = np.array(replaced_panels)

            
        if action > 0:
            step_CAPEX, panel_cost_and_inverter = self.calculate_total_CAPEX(actions_step, self.panel_cost_and_inverter)
            self.pv_costs -= step_CAPEX
            
        else:
            panel_cost_and_inverter = 0
                
        next_observation = self._get_obs()

        # Calculate the Reslae value
        resale = self.calculate_resale(panel_cost_and_inverter, indices) #  ***
        
        self.pv_costs += resale

        
        # CALCULATE THE BUDGET INTEREST
        current_penalty, due_loans, next_step_penalty = self.calculate_penalty(self.episode_len, self.pv_costs)
        
        
        # CALCULATE THE ENERGY YIELD
        exported_revenue, AC_OUTPUT_tot, minimised_revenue = self.calculate_import_export(self.AC_OUTPUT, 
                                                                          self.elec_df, FiT, self.import_price)        
        
        pv_costs_observation = - self.pv_costs / 10000
        self.observation[self.number_of_panels + 4] = pv_costs_observation
        
        next_step_penalty_observation = - next_step_penalty / 8000
        self.observation[self.number_of_panels + 5] = next_step_penalty_observation
        
        
        # CALCULATE STEP BALANCES
        self.fin_balance += self.pv_costs
        self.fin_balance += current_penalty
        self.fin_balance += float(exported_revenue + minimised_revenue)
        
        # CALCULATE TOTAL BALANCES
        self.fin_balance_tot += self.fin_balance                
        
        # SUBSTRACT 1 FOR TIMESTEP
        self.episode_len -= 1
        done = self.episode_len <= 0
        
        # CALCULATE IRR, NPV AND CARBON INTENSITY
        total_cash_flow = self.calculate_irr_and_npv(self.pv_costs, exported_revenue, minimised_revenue, current_penalty)
        irr = npf.irr(total_cash_flow) * 100
        npv = npf.npv(0.04 ,total_cash_flow)
            
        # RETURNS AND CALCULATE REWARD
        if self.episode_len == 0:
            irr_fin = irr
            npv_fin = npv
        
        reward = self.fin_balance / 1000
        #reward = self.fin_balance_tot / 1000 if done else 0
        
        # FAILURE
         
        survival = self.failure(actions_step)
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)

        for c, p in enumerate(survival):
            
            if c < self.number_of_panels:

                if p - 1 <= abs(self.episode_len - 24):
                    self.broke[c] = 1

                    self.observation[c] = 0
        
        # DEGRADATION RATE
        # Applying degradation only to panels that are operational (above 0.1 efficiency)
        active_panels = self.observation[:self.number_of_panels] > 0.1
        degradations = np.random.normal(self.deg_mu, self.deg_std, size=self.number_of_panels) / 100
        self.observation[:self.number_of_panels][active_panels] -= degradations[active_panels]
        
        if not done: 
        
            self.next_cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
            self.next_import_price = self.random_import_price[abs(self.episode_len - 25)]
            self.next_efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
            next_FIT = self.calculate_FiT(self.episode_len, self.next_import_price)
        
            price_observation = (self.next_import_price - 0.00022499) / (0.0020798 - 0.00022499)
            self.observation[self.number_of_panels] = price_observation

            FiT_observation = (next_FIT - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
            self.observation[self.number_of_panels + 1] = FiT_observation

            eff_observation = (self.next_efficency_develop - 0.999) / (1.156 - 0.999)
            self.observation[self.number_of_panels + 2] = eff_observation

            cost_per_Wp_observation = (self.next_cost_per_Wp - 0.254) / (1.12 - 0.254) 
            self.observation[self.number_of_panels + 3] = cost_per_Wp_observation
        
        
        info = {"step financial balance (eur):": self.fin_balance,
               "total financial balance: (eur)": self.fin_balance_tot,
               "internal rate of return": irr_fin,
               "current_interest": current_penalty,
                "net present value": npv_fin}
         
        
        self.previous_observation = self.observation.copy()
        
        return self.observation, reward, done, False, info

In [5]:
env = TrainEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_train_arr, Eff_train_arr, CAPEX_JA_train_arr)
env_test = TestEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_test_arr, Eff_test_arr, CAPEX_JA_test_arr)

In [13]:
"""
class RewardNormalizer:
    def __init__(self):
        self.n = 0
        self.mean = 0
        self.M2 = 0

    def update(self, reward):
        self.n += 1
        delta = reward - self.mean
        self.mean += delta / self.n
        delta2 = reward - self.mean
        self.M2 += delta * delta2

    def normalize(self, reward):
        if self.n < 2:
            return reward  # Not enough data to normalize
        variance = self.M2 / (self.n - 1)
        std_dev = variance ** 0.5
        return (reward - self.mean) / std_dev 

    def sample_and_normalize_rewards(self, episodes, environment):
        normalized_rewards = []
        rewards = []
        
        for episode in range(episodes):
            done = False
            obs = environment.reset()
            step = 0
            
            while not done:
                step += 1
                random_action = environment.action_space.sample()
                obs, reward, done, _, info = environment.step(random_action)
                
                values = list(info.values())

                reward = values[0]

                # Assuming the reward to normalize is directly obtained from the environment step
                self.update(reward)
                normalized_reward = self.normalize(reward)
                normalized_rewards.append(normalized_reward)
                rewards.append(reward)
        
        # Calculate the final standard deviation
        final_std_dev = (self.M2 / (self.n - 1)) ** 0.5 if self.n > 1 else 0

        return normalized_rewards, rewards, self.mean, final_std_dev

normalizer = RewardNormalizer()
normalized_rewards, rewards, final_mean, final_std_dev = normalizer.sample_and_normalize_rewards(1000, env)
"""

C:\Users\kubaw\AppData\Local\Temp\ipykernel_12144\3868030579.py:178: RuntimeWarning: divide by zero encountered in scalar divide
  value = annual_expense / self.current_budget_constraint


In [43]:
check_env(env)

In [1]:
test3(1, env)

In [2]:
test1(1000, env)

In [15]:
def make_env(rank: int, seed: int = 0) -> Callable:
    def _init() -> gym.Env:
        random.seed(seed + rank)
        np.random.seed(seed + rank) 
        env = TrainEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_train_arr, Eff_train_arr, CAPEX_JA_train_arr)
        env.reset(seed=seed + rank)
        return env

    return _init
# Number of environments to run in parallel
num_cpu = 16
env = SubprocVecEnv([make_env(i) for i in range(num_cpu)])   

In [16]:
import math
def logarithmic_schedule(initial_value, final_value=0.00001):
    """
    Returns a function that computes a logarithmically decreasing value from initial_value to final_value.
    """
    def func(progress_remaining):
        # Avoid taking log of zero by setting a lower limit close to zero
        epsilon = 0.0001
        progress = max(epsilon, 1 - progress_remaining)
        # Calculate the decay factor using a logarithmic scale
        return final_value + (initial_value - final_value) * math.log(1/progress)
    return func


learning_rate = logarithmic_schedule(0.0001)

In [17]:
log_path = "./logs/"
eval_callback = EvalCallback(env_test, best_model_save_path = "C:/Users/kubaw/Desktop/DELFT/THESIS/CODE/TEST_MODELS/ja24_low/",
                             log_path = log_path, n_eval_episodes = 750, eval_freq=10000,
                             deterministic=True, render=False)


In [18]:
policy_kwargs = dict(net_arch=dict(pi=[1024, 1024], vf=[1024, 1024]))

In [19]:
def linear_schedule(initial_value, final_value=0.00001):
    """
    Returns a function that computes a linearly decreasing value from initial_value to final_value.
    """
    def func(progress_remaining):
        # Calculate the decrease based on the remaining progress
        return final_value + (initial_value - final_value) * progress_remaining
    return func

# Define the learning rate using the linear schedule
learning_rate = linear_schedule(0.0003)

In [20]:
model = PPO("MlpPolicy", env, learning_rate = learning_rate, batch_size = 1024, n_epochs = 24, policy_kwargs = policy_kwargs, gamma = 0.99,  verbose=1, tensorboard_log = "C:/Users/kubaw/Desktop/DELFT/THESIS\CODE/TEST_MODELS/LOGS/logs")
TIMESTEPS = 10000000
model.learn(total_timesteps = TIMESTEPS, callback=eval_callback)

Using cpu device
Logging to C:/Users/kubaw/Desktop/DELFT/THESIS\CODE/TEST_MODELS/LOGS/logs\PPO_387


C:\Users\kubaw\miniforge3\envs\pytorch-env\lib\site-packages\stable_baselines3\common\callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x00000167E1791B70> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x00000167E17933D0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


------------------------------
| time/              |       |
|    fps             | 3921  |
|    iterations      | 1     |
|    time_elapsed    | 8     |
|    total_timesteps | 32768 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1189        |
|    iterations           | 2           |
|    time_elapsed         | 55          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.016887046 |
|    clip_fraction        | 0.292       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.2        |
|    explained_variance   | -0.000337   |
|    learning_rate        | 0.000299    |
|    loss                 | 27.6        |
|    n_updates            | 24          |
|    policy_gradient_loss | -0.0261     |
|    value_loss           | 71          |
-----------------------------------------
---------------------------

C:\Users\kubaw\miniforge3\envs\pytorch-env\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\kubaw\AppData\Local\Temp\ipykernel_3616\3881792178.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  value = annual_expense / self.current_budget_constraint


Eval num_timesteps=160000, episode_reward=27.08 +/- 13.59
Episode length: 25.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 25          |
|    mean_reward          | 27.1        |
| time/                   |             |
|    total_timesteps      | 160000      |
| train/                  |             |
|    approx_kl            | 0.017697046 |
|    clip_fraction        | 0.355       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.99       |
|    explained_variance   | 0.607       |
|    learning_rate        | 0.000296    |
|    loss                 | 36.5        |
|    n_updates            | 96          |
|    policy_gradient_loss | -0.0356     |
|    value_loss           | 67.5        |
-----------------------------------------
New best mean reward!
-------------------------------
| time/              |        |
|    fps             | 675    |
|    iterations      | 5      |
|    

...

--------------------------------
| time/              |         |
|    fps             | 539     |
|    iterations      | 84      |
|    time_elapsed    | 5099    |
|    total_timesteps | 2752512 |
--------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 539         |
|    iterations           | 85          |
|    time_elapsed         | 5158        |
|    total_timesteps      | 2785280     |
| train/                  |             |
|    approx_kl            | 0.006319752 |
|    clip_fraction        | 0.0628      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.932      |
|    explained_variance   | 0.664       |
|    learning_rate        | 0.00022     |
|    loss                 | 20.6        |
|    n_updates            | 2016        |
|    policy_gradient_loss | -0.00196    |
|    value_loss           | 44.9        |
-----------------------------------------
-------------

-----------------------------------------
| time/                   |             |
|    fps                  | 537         |
|    iterations           | 95          |
|    time_elapsed         | 5796        |
|    total_timesteps      | 3112960     |
| train/                  |             |
|    approx_kl            | 0.004878043 |
|    clip_fraction        | 0.0484      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.74       |
|    explained_variance   | 0.688       |
|    learning_rate        | 0.000211    |
|    loss                 | 22.2        |
|    n_updates            | 2256        |
|    policy_gradient_loss | -0.00187    |
|    value_loss           | 39.7        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 537         |
|    iterations           | 96          |
|    time_elapsed         | 5854        |
|    total_timesteps      | 314572

-----------------------------------------
| time/                   |             |
|    fps                  | 535         |
|    iterations           | 105         |
|    time_elapsed         | 6424        |
|    total_timesteps      | 3440640     |
| train/                  |             |
|    approx_kl            | 0.004869872 |
|    clip_fraction        | 0.0501      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.67       |
|    explained_variance   | 0.696       |
|    learning_rate        | 0.000201    |
|    loss                 | 19.2        |
|    n_updates            | 2496        |
|    policy_gradient_loss | -0.00244    |
|    value_loss           | 40.6        |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 535          |
|    iterations           | 106          |
|    time_elapsed         | 6482         |
|    total_timesteps      | 3

------------------------------------------
| time/                   |              |
|    fps                  | 533          |
|    iterations           | 115          |
|    time_elapsed         | 7058         |
|    total_timesteps      | 3768320      |
| train/                  |              |
|    approx_kl            | 0.0035918856 |
|    clip_fraction        | 0.0436       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.609       |
|    explained_variance   | 0.691        |
|    learning_rate        | 0.000192     |
|    loss                 | 19.4         |
|    n_updates            | 2736         |
|    policy_gradient_loss | -0.00193     |
|    value_loss           | 41.7         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 534          |
|    iterations           | 116          |
|    time_elapsed         | 7112         |
|    total_

KeyboardInterrupt: 

In [21]:
model.save(r"C:\Users\kubaw\Desktop\DELFT\THESIS\CODE\TEST_MODELS\FINAL24_ja_low")

In [10]:
model = PPO.load(r"C:\Users\kubaw\Desktop\DELFT\THESIS\CODE\TEST_MODELS\FINAL24_ja_low.zip")

In [11]:
evaluate1(1, env_test, model)

Act: 9 
 Obs: [0.8506809  0.8584513  0.9979332  0.9444468  0.94620717 0.8702701
 0.89418256 0.9858364  0.99208087 0.         0.         0.9894404
 0.9902745  0.9319078  0.9669751  0.930064   0.9864388  0.9897222
 0.8567538  0.99934334 0.9690382  0.9883636  0.98566294 0.96554
 0.07780816 0.08083121 0.04069865 0.6843568  0.22262634 0.06385198
 0.4304    ] 
 Balance 934.6484733204852
Act: 1 
 Obs: [0.83448714 0.84543145 0.9839316  0.92941517 0.9384754  0.8553898
 0.88254833 0.9720743  0.97176784 0.99386346 0.         0.98887855
 0.9821276  0.918809   0.9589939  0.9213087  0.971079   0.98773867
 0.8422795  0.9796777  0.9611708  0.97831863 0.9790401  0.9443
 0.0921257  0.08930569 0.04105865 0.6808497  0.09113253 0.0676831
 0.9064    ] 
 Balance 1145.615082726474
Act: 3 
 Obs: [0.98640937 0.8348458  0.97125524 0.91823715 0.92682475 0.8490568
 0.8809269  0.96460253 0.964308   0.98867255 0.9894532  0.97657263
 0.974911   0.9099361  0.9591434  0.9067026  0.9565509  0.9706006
 0.9933011  0.96924

In [12]:
def evaluate2(episodes, environment, model):
    
    mean_irr = 0
    mean_fin_balance = 0
    irr = 0
    fin_balance = 0
    count = 0
    npv = 0
    list_npv = []

    for ep in range(episodes):

        obs, _ = environment.reset()  # Unpack the tuple and ignore the info part
        done = False

        while not done:
            action, _ = model.predict(obs)  # Now obs is just the observation array
            obs, reward, done, truncated, info = environment.step(action)
            # Extracting the 2nd and 3rd key-value pairs
            keys = list(info.keys())
            values = list(info.values())

            # Getting the 2nd key-value pair
            second_value = values[1]

            # Getting the 3rd key-value pair
    
            third_value = values[2]
            fourth_value = values[4]
        
        fin_balance += second_value
        npv += fourth_value
        count += 1
        
        list_npv.append(fourth_value)
            
    mean_fin_balance = fin_balance/count
    mean_npv = npv/count

    #print(mean_npv)

    environment.close()
    
    return(list_npv)

In [59]:
def basepolicy1(episodes, environment):
    
    mean_irr = 0
    mean_fin_balance = 0
    irr = 0
    fin_balance = 0
    count = 0
    irr_count = 0
    npv = 0
    list_npv = []

    for ep in range(episodes):

        obs, _ = environment.reset()  # Unpack the tuple and ignore the info part
        done = False

        while not done:
            
            action = 0
            for i, n in enumerate(obs):
                if i < 24:
                    if n < 0.75:
                        action += 1

            obs, reward, done, truncated, info = environment.step(action)

            # Extracting the 2nd and 3rd key-value pairs
            keys = list(info.keys())
            values = list(info.values())

            # Getting the 2nd key-value pair
            second_value = values[1]

            # Getting the 3rd key-value pair
    
            third_value = values[2]
            fourth_value = values[4]
        
        fin_balance += second_value
        npv += fourth_value
        count += 1
        
        list_npv.append(fourth_value)
            
    mean_fin_balance = fin_balance/count
    mean_npv = npv/count

    #print(mean_npv, "\n", mean_irr, "\n" )

    environment.close()
    
    return(list_npv)

In [60]:
values_policy = basepolicy1(1000, env_test)

C:\Users\kubaw\AppData\Local\Temp\ipykernel_24352\2392353474.py:197: RuntimeWarning:

divide by zero encountered in scalar divide



In [191]:
class TestEnvironment(gym.Env):
    def __init__(self, AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_tariff, efficency, CAPEX, seed):
        
        # Price per watthour
        self.import_price_df = import_tariff
        self.import_price_at_zero = np.float32(0.00035)
        self.import_price_rate = import_price_rate
        
        # Energy Balance
        self.AC_OUTPUT = AC_OUTPUT_arr
        self.elec_df = elec_consum_arr
        self.max_export = 4000
        self.number_of_panels = 24
        
        self.seed = seed
        np.random.seed(self.seed)  # Seed numpy's RNG for deterministic results
        
        # Degradation
        self.deg_mu = 1.055
        self.deg_std = 0.555
        
        # Efficency Development
        self.efficency_develop_df = efficency
        self.efficency_at_zero = 1.0
        
        # Costs
        self.power_at_zero = 415
        self.cost_per_Wp_df_at_zero = 0.69
        self.cost_per_Wp_df = CAPEX
        self.initial_other_costs = 150
        
        self.operational_cost = 16.8
        #self.budget_constraint = 1500
        
        self.loan_interest_rate = 1.06
        self.normal_interest_rate = 1.02
                        
        # Spaces and length
        self.action_space = spaces.Discrete(self.number_of_panels + 1)
        self.observation_space = spaces.Box(0, 1.25, shape=(self.number_of_panels + 7,))
        self.episode_len = 25
        self.months_per_timestep = 12
        
    def _get_obs(self):
        
        return self.observation
    
    def calculate_import_export(self, AC_OUTPUT, elec_df, export_price, import_price):
        
        """
        Calculate the annual Wh of energy exported to the grid (exported) and saved (minimised)
        """
        
        AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT 

        exported = (AC_OUTPUT_tot - self.elec_df).clip(min=0, max = self.max_export)        
        export_revenue = (export_price * exported).sum()

        
        minimised = AC_OUTPUT_tot - exported 
        minimised_revenue = (minimised * (self.import_price_rate * import_price)).sum()
        

        return export_revenue, AC_OUTPUT_tot, minimised_revenue
    
    def reset(self, seed=None):
        
        """
        Reset the environment to the original state at t=1
        """
        if seed is not None:
            self.seed = seed
        np.random.seed(self.seed)
                
        # Panels
        self.init_obs = np.random.choice([1, 0], size=self.number_of_panels, p=[0.4, 0.6]).astype(np.float32)
        self.init_obs = np.where(self.init_obs < 0.5, 0.0, np.random.uniform(0.85, 1.0, size=self.number_of_panels))

        # Combine all initialization into a single step for efficiency
        self.import_price_at_zero_norm = (self.import_price_at_zero - 0.00022499) / (0.0020798 - 0.00022499)
        self.FiT_at_zero_norm = (self.import_price_at_zero - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
        self.efficency_at_zero_norm = (self.efficency_at_zero - 0.999) / (1.156 - 0.999)
        self.panel_cost_and_inverter_at_zero_norm = (self.cost_per_Wp_df_at_zero - 0.254) / (1.12 - 0.254)
        
        self.current_budget_constraint = np.random.randint(750, 2000)
        self.next_step_budget_constraint = 0
        
        
        # Complete observation initialization in one go
        self.observation = np.concatenate([
            self.init_obs,
            [self.import_price_at_zero_norm, self.FiT_at_zero_norm, self.efficency_at_zero_norm, 
             self.panel_cost_and_inverter_at_zero_norm, 0., 0., 0.]
        ]).astype(np.float32)

        self.previous_observation = self.observation.copy()

        # RANDOM IMPORT PRICE
        self.random_import_price = self.import_price_df[np.random.choice(self.import_price_df.shape[0])] 

        # RANDOM EFFICENCY
        self.random_efficency_develop = self.efficency_develop_df[np.random.choice(self.efficency_develop_df.shape[0])]   
        
        # RANDOM COST PER WP
        self.random_cost_per_Wp = self.cost_per_Wp_df[np.random.choice(self.cost_per_Wp_df.shape[0])]   
        
        
        self.episode_len = 25  
    
        info = {}
        
        # RESET BALANCES
        self.fin_balance_tot = 0
        self.reward_tot = 0
        self.env_balance_tot = 0
        self.produced = 0
        self.other_costs = 0
        self.FiT = 0.0004
        self.next_FiT = 0.0004
        self.resale_values = array_of_zeros = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        self.total_cash_flow = []
        self.annual_cash_flow = 0
                
        self.due_loans = [0, 0, 0, 0] 
        self.current_interest = 0
        self.step_total_interest = 1
        
        self.two_year_ago_interest = 0
        self.first_year_interest = []
        self.second_year_interest = [0]
        self.third_year_interest = [0, 0]
        self.fourth_year_interest = [0, 0, 0]
        self.next_year_total = 0
        
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
    
        return self.observation, info
    
    def calculate_resale(self, initial_panel_cost, indices):
        
        self.resale_values[indices] = initial_panel_cost
        
        self.resale_values = self.resale_values * 0.85
        
        for count, i in enumerate(self.broke):
            if i == 1:
                self.resale_values[count] = 0
        
        resale_step = self.resale_values[indices].sum()
        
        return resale_step
    
    def calculate_panel_inv_cost(self, cost_per_Wp):
        
        PW_ep = self.efficency_develop * self.power_at_zero
        
        panel_cost_and_inverter = PW_ep * cost_per_Wp
        
        return panel_cost_and_inverter
    
    def calculate_irr_and_npv(self, pv_cost, minimised_revenue, export_revenue, penalty):
                
        """
        Calculates total cash flow of the project needed for the internal rate of return
        """ 
        self.expences = 0
        self.annual_cash_flow = 0
        initial_cost = 0
        
        self.expences = pv_cost
        self.annual_cash_flow = self.expences + export_revenue + minimised_revenue + penalty
        initial_cost_q, x = self.calculate_total_CAPEX(self.init_obs, self.panel_cost_and_inverter)
        initial_cost = - initial_cost_q
        
        if self.episode_len == 24:
            self.total_cash_flow.append(initial_cost + self.annual_cash_flow) 
        else:
            self.total_cash_flow.append(self.annual_cash_flow) 
        
        return self.total_cash_flow
        
    def calculate_penalty(self, current_step, annual_expense):
              
        year = 25 - current_step
        
        if year > 0:
            self.current_budget_constraint = self.next_step_budget_constraint    
            
        
        self.current_interest = self.next_year_total
        annual_expense = (-annual_expense)
        value = 0 
        loan = 0
        annual_interest = 0

        if annual_expense > self.current_budget_constraint:
            loan = (self.current_budget_constraint - annual_expense)
            value = annual_expense / self.current_budget_constraint
            periods = 2 if value < 2 else 3 if value < 3 else 4

            annual_interest = loan / periods
            interest_multiplier = 1

            for i in range(4):
                if i < periods:
                    self.due_loans[i] = annual_interest * interest_multiplier
                    interest_multiplier *= self.loan_interest_rate
                else:
                    self.due_loans[i] = 0
        else:
             self.due_loans = [0, 0, 0, 0]
    
        self.first_year_interest.append(self.due_loans[0])
        self.second_year_interest.append(self.due_loans[1])
        self.third_year_interest.append(self.due_loans[2])
        self.fourth_year_interest.append(self.due_loans[3])
    
    
        self.next_year_total = self.first_year_interest[year] + self.second_year_interest[year] + self.third_year_interest[year] + self.fourth_year_interest[year]
        
        self.next_step_budget_constraint = np.random.randint(750, 2000) * self.step_total_interest
        current_budget_observation = (self.next_step_budget_constraint - 750 * self.step_total_interest) / (2000 * self.step_total_interest - 750 * self.step_total_interest) 
        self.observation[self.number_of_panels + 6] = current_budget_observation
                
        return self.current_interest, self.due_loans, self.next_year_total
        
    def calculate_total_CAPEX(self, action_step, panel_cost_and_inverter):
        """
        Calculate CAPEX each step in a vectorized manner.
        """
        BOS = panel_cost_and_inverter * 0.55
        number_installed = int(np.sum(action_step))

        # Calculate costs from module and inverter
        panel_cost_and_inverter_step = panel_cost_and_inverter * number_installed

        # Calculate other installation costs
        if number_installed == 0:
            other_costs = 0
        elif number_installed == 1:
            other_costs = self.initial_other_costs * self.step_total_interest
        else:
            discounts = 0.9 ** np.arange(number_installed)
            other_costs = (self.initial_other_costs * self.step_total_interest * discounts).sum()

        # Calculate BOS costs using vector operations
        is_new_installation = (self.previous_observation[:number_installed] == 0) & (action_step[:number_installed] == 1)
        is_replacement = (self.previous_observation[:number_installed] > 0) & (action_step[:number_installed] == 1)
        BOS_cost = np.sum(BOS * is_new_installation) + np.sum((BOS / 2) * is_replacement)

        # Sum total CAPEX
        total_CAPEX = panel_cost_and_inverter_step + BOS_cost + other_costs

        return total_CAPEX, panel_cost_and_inverter
        
    def failure(self, actions):
        
        beta = 3  # Shape parameter
        phi = 30  # Scale parameter  

        # Determine which panels are active based on the actions and previous observations.
        if self.episode_len == 24:
            active_panels = (self.observation[:self.number_of_panels] > 0.85)
        else:
            active_panels = (self.observation[:self.number_of_panels] == self.efficency_develop)

        # Calculate lifespan for all active panels at once
        lifespans = np.random.weibull(beta, self.number_of_panels) * phi
        lifespans = np.where(active_panels, lifespans, 0)  # Apply lifespan only to active panels

        # Adjust survival times based on episode length
        self.survival[:self.number_of_panels] = np.where(
            active_panels,
            np.abs(lifespans.astype(int)) + np.abs(self.episode_len - 25),
            self.survival[:self.number_of_panels]
        )

        return self.survival

    def calculate_FiT(self, episodes, import_price):
            
        self.FiT = import_price
            
        if episodes == 25:
            self.FiT = self.FiT
            
        elif episodes == 24 or episodes == 23:
            self.FiT = self.FiT * 0.64
            
        elif episodes == 22:
            self.FiT = self.FiT * 0.46
            
        elif episodes == 21:
            self.FiT = self.FiT * 0.55
            
        elif episodes < 20:
            self.FiT = self.FiT * 0.33
            
        elif episodes == 20:
            self.FiT = self.FiT * 0.37
            
        return self.FiT
                        
    def step(self, action):
        
        """
        defines actions, reward etc.
        """
        
        # RESET THE ANNUAL BALANCES
        self.total_CAPEX = 0
        self.pv_costs = 0
        self.fin_balance = 0
        self.number_installed = 0
        irr_fin = 0
        npv_fin = 0
        current_penalty = 0
        self.other_costs = 0
        next_step_penalty = 0
        self.step_total_interest = self.step_total_interest * self.normal_interest_rate
        current_operational_costs = self.operational_cost * self.step_total_interest
        
        
        self.cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
        self.import_price = self.random_import_price[abs(self.episode_len - 25)]
        self.efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
           
        self.panel_cost_and_inverter = self.calculate_panel_inv_cost(self.cost_per_Wp)
        FiT = self.calculate_FiT(self.episode_len, self.import_price)
        
        reward = 0   
        actions_step = np.random.rand(self.number_of_panels + 1)
        
        
        # Find indices of the lowest 'action' values in previous_observation
        indices = np.argsort(self.previous_observation[:self.number_of_panels])[:action]

        # Replace these indices in the observation with efficiency_develop
        self.observation[:self.number_of_panels][indices] = self.efficency_develop

        # Copy over the other values from previous_observation to observation
        mask = np.ones(len(self.previous_observation[:self.number_of_panels]), dtype=bool)
        mask[indices] = False
        self.observation[:self.number_of_panels][mask] = self.previous_observation[:self.number_of_panels][mask]

        replaced_panels = np.zeros(len(self.previous_observation[:self.number_of_panels]), dtype=int)
        replaced_panels[indices] = 1

        instaltion = (self.observation[:self.number_of_panels] > 0).astype(int)
        self.pv_costs -= instaltion.sum() * current_operational_costs

        actions_step = np.array(replaced_panels)

            
        if action > 0:
            step_CAPEX, panel_cost_and_inverter = self.calculate_total_CAPEX(actions_step, self.panel_cost_and_inverter)
            self.pv_costs -= step_CAPEX
            
        else:
            panel_cost_and_inverter = 0
                
        next_observation = self._get_obs()

        # Calculate the Reslae value
        resale = self.calculate_resale(panel_cost_and_inverter, indices) #  ***
        
        self.pv_costs += resale

        
        # CALCULATE THE BUDGET INTEREST
        current_penalty, due_loans, next_step_penalty = self.calculate_penalty(self.episode_len, self.pv_costs)
        
        
        # CALCULATE THE ENERGY YIELD
        exported_revenue, AC_OUTPUT_tot, minimised_revenue = self.calculate_import_export(self.AC_OUTPUT, 
                                                                          self.elec_df, FiT, self.import_price)        
        
        pv_costs_observation = - self.pv_costs / 10000
        self.observation[self.number_of_panels + 4] = pv_costs_observation
        
        next_step_penalty_observation = - next_step_penalty / 8000
        self.observation[self.number_of_panels + 5] = next_step_penalty_observation
        
        
        # CALCULATE STEP BALANCES
        self.fin_balance += self.pv_costs
        self.fin_balance += current_penalty
        self.fin_balance += float(exported_revenue + minimised_revenue)
        
        # CALCULATE TOTAL BALANCES
        self.fin_balance_tot += self.fin_balance                
        
        # SUBSTRACT 1 FOR TIMESTEP
        self.episode_len -= 1
        done = self.episode_len <= 0
        
        # CALCULATE IRR, NPV AND CARBON INTENSITY
        total_cash_flow = self.calculate_irr_and_npv(self.pv_costs, exported_revenue, minimised_revenue, current_penalty)
        irr = npf.irr(total_cash_flow) * 100
        npv = npf.npv(0.04 ,total_cash_flow)
            
        # RETURNS AND CALCULATE REWARD
        if self.episode_len == 0:
            irr_fin = irr
            npv_fin = npv
        
        reward = self.fin_balance / 1000
        #reward = self.fin_balance_tot / 1000 if done else 0
        
        # FAILURE
         
        survival = self.failure(actions_step)
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)

        for c, p in enumerate(survival):
            
            if c < self.number_of_panels:

                if p - 1 <= abs(self.episode_len - 24):
                    self.broke[c] = 1

                    self.observation[c] = 0
        
        # DEGRADATION RATE
        # Applying degradation only to panels that are operational (above 0.1 efficiency)
        active_panels = self.observation[:self.number_of_panels] > 0.1
        degradations = np.random.normal(self.deg_mu, self.deg_std, size=self.number_of_panels) / 100
        self.observation[:self.number_of_panels][active_panels] -= degradations[active_panels]
        
        if not done: 
        
            self.next_cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
            self.next_import_price = self.random_import_price[abs(self.episode_len - 25)]
            self.next_efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
            next_FIT = self.calculate_FiT(self.episode_len, self.next_import_price)
        
            price_observation = (self.next_import_price - 0.00022499) / (0.0020798 - 0.00022499)
            self.observation[self.number_of_panels] = price_observation

            FiT_observation = (next_FIT - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
            self.observation[self.number_of_panels + 1] = FiT_observation

            eff_observation = (self.next_efficency_develop - 0.999) / (1.156 - 0.999)
            self.observation[self.number_of_panels + 2] = eff_observation

            cost_per_Wp_observation = (self.next_cost_per_Wp - 0.254) / (1.12 - 0.254) 
            self.observation[self.number_of_panels + 3] = cost_per_Wp_observation
        
        
        info = {"budget:": self.current_budget_constraint,
               "import price": self.import_price,
               "export price": FiT,
               "PV_panel_price": self.panel_cost_and_inverter,
                "pv array size": (self.observation[:self.number_of_panels]).sum() * self.power_at_zero * self.efficency_develop,
               "current_interest": current_penalty,
               "efficency_develop": self.efficency_develop,
               "action": action}
         
        
        self.previous_observation = self.observation.copy()
        
        return self.observation, reward, done, False, info

In [226]:
env_test2 = TestEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_test_arr, 
                            Eff_test_arr, CAPEX_JA_test_arr, 24)

#4790

In [227]:
def evaluate_visual(episodes, environment, model):
    
    for ep in range(episodes):

        obs, _ = environment.reset()  # Unpack the tuple and ignore the info part
        done = False
        
        budget_arr = []
        import_price_arr = []
        export_price_arr = []
        PV_panel_price_arr = []
        pv_array_size_arr = []
        current_interest_arr = []
        efficency_develop_arr = []
        action_arr = []
        rewards = 0
        

        while not done:
            action, _ = model.predict(obs)  # Now obs is just the observation array
            obs, reward, done, truncated, info = environment.step(action)
            # Extracting the 2nd and 3rd key-value pairs
            values = list(info.values())

            # Getting the 2nd key-value pair
            budget = values[0]
            budget_arr.append(budget)
            
            import_price = values[1]
            import_price_arr.append(import_price)
            
            export_price = values[2]
            export_price_arr.append(export_price)
            
            PV_panel_price = values[3]
            PV_panel_price_arr.append(PV_panel_price)
            
            Ppv_array_size = values[4]
            pv_array_size_arr.append(Ppv_array_size)
            
            current_interest = values[5]
            current_interest_arr.append(current_interest)
            
            efficency_develop = values[6]
            efficency_develop_arr.append(efficency_develop)   
            
            action = values[7]
            action_arr.append(action)
            
            rewards += reward
            

    environment.close()          
    
    print(rewards)
    
    return budget_arr, import_price_arr, export_price_arr, PV_panel_price_arr, pv_array_size_arr, current_interest_arr, efficency_develop_arr, action_arr

In [228]:
budget_arr, import_price_arr, export_price_arr, PV_panel_price_arr, pv_array_size_arr, current_interest_arr, efficency_develop_arr, action_arr = evaluate_visual(1, env_test2, model)

44.97240286590212


In [229]:
def basepolicy2(episodes, environment):
    
    pv_array_size_arr = []
    action_arr = []
    current_interest_arr = []
    rewards = 0

    for ep in range(episodes):

        obs, _ = environment.reset()  # Unpack the tuple and ignore the info part
        done = False

        while not done:
            
            action = 0
            for i, n in enumerate(obs):
                if i < 24:
                    if n < 0.8:
                        action += 1

            obs, reward, done, truncated, info = environment.step(action)

            # Extracting the 2nd and 3rd key-value pairs
            values = list(info.values())

            Ppv_array_size = values[4]
            pv_array_size_arr.append(Ppv_array_size)

            # Getting the 3rd key-value pair
    
            action = values[7]
            action_arr.append(action)
            
            current_interest = values[5]
            current_interest_arr.append(current_interest)
            
            rewards += reward

    print(rewards)
    environment.close()
    return pv_array_size_arr, action_arr, current_interest_arr

In [230]:
Ppv_array_size_policy, action_arr_policy, current_interest_arr_policy  = basepolicy2(1, env_test2)

40.72972577650635


In [231]:
#bla 
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a figure with subplots (7 rows, 1 column)
fig = make_subplots(rows=9, cols=1)

# Add line plots for the first 6 arrays in separate rows
fig.add_trace(go.Scatter(y=budget_arr, mode='lines', name='Budget'), row=1, col=1)
fig.add_trace(go.Scatter(y=import_price_arr, mode='lines', name='Import Price'), row=2, col=1)
fig.add_trace(go.Scatter(y=export_price_arr, mode='lines', name='Export Price'), row=3, col=1)
fig.add_trace(go.Scatter(y=PV_panel_price_arr, mode='lines', name='Panel CAPEX'), row=4, col=1)
fig.add_trace(go.Scatter(y=efficency_develop_arr, mode='lines', name='Efficency'), row=5, col=1)
fig.add_trace(go.Scatter(y=current_interest_arr, mode='lines', name='Interest'), row=6, col=1)
fig.add_trace(go.Scatter(y=current_interest_arr_policy, mode='lines', name='Interest'), row=7, col=1)
# Add a bar plot for the 7th array in the last row
fig.add_trace(go.Bar(y=pv_array_size_arr, name='Array Size _agent'), row=8, col=1)

# Add a bar plot for the 7th array in the last row
fig.add_trace(go.Bar(y=Ppv_array_size_policy, name='Array Size _policy'), row=9, col=1)

# Update layout for clarity
fig.update_layout(title="Line and Bar Plots in Separate Subplots",
                  xaxis_title="Index",
                  yaxis_title="Value",
                  height=1400,  # Increase height to accommodate all subplots
                  showlegend=True)

# Update the y-axis range for the 7th subplot to start from 12
fig.update_yaxes(range=[6000, max(pv_array_size_arr) + 1], row=8, col=1)
fig.update_yaxes(range=[6000, max(Ppv_array_size_policy) + 1], row=9, col=1)

# Show the figure
fig.show()

In [233]:
action_arr, action_arr_policy

([array(13, dtype=int64),
  array(1, dtype=int64),
  array(2, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(2, dtype=int64),
  array(1, dtype=int64),
  array(2, dtype=int64),
  array(2, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(2, dtype=int64),
  array(1, dtype=int64),
  array(4, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(1, dtype=int64),
  array(2, dtype=int64),
  array(1, dtype=int64),
  array(3, dtype=int64),
  array(1, dtype=int64)],
 [15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 2, 1, 1, 3, 3, 3, 3, 0, 2, 1, 2])